<a href="https://colab.research.google.com/github/dssp-ethz/DSSP-Illustrations/blob/main/LMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>